In [1]:
import pandas as pd
import numpy as np
import os
import joblib
import pickle

In [2]:
feat_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/merged_features/features_sparse'
vocab_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/merged_features/vocab'
analysis_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/features_by_analysis'
cohort_path = '/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort'

In [3]:
def read_file(filename, columns=None, **kwargs):
    print(filename)
    load_extension = os.path.splitext(filename)[-1]
    if load_extension == ".parquet":
        return pd.read_parquet(filename, columns=columns,**kwargs)
    elif load_extension == ".csv":
        return pd.read_csv(filename, usecols=columns, **kwargs)

In [4]:
feats_id_map = read_file(
		os.path.join(
			feat_path,
			"features_row_id_map.parquet"
		),
		engine='pyarrow'
	)
vocab = read_file(
		os.path.join(
			vocab_path,
			"vocab.parquet"
		),
		engine='pyarrow'
)

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/merged_features/features_sparse/features_row_id_map.parquet
/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/merged_features/vocab/vocab.parquet


In [5]:
features = joblib.load(os.path.join(feat_path,"features.gz"))

In [6]:
cohort = read_file(
		os.path.join(
			cohort_path,
			"cohort_split.parquet"
		),
		engine='pyarrow'
	)
cohort = cohort.merge(feats_id_map)

/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/cohort/cohort_split.parquet


In [7]:
ped_df = cohort[cohort['age_group'] == '<18']

In [8]:
adult_df = cohort[cohort['age_group'] != '<18']

In [9]:
ped_rows = list(ped_df['features_row_id'])
ad_rows = list(adult_df['features_row_id'])

ped_row_map = pd.DataFrame({'row_idx_new':[i for i in range(len(ped_rows))], 'row_idx_og':ped_rows})
ad_row_map = pd.DataFrame({'row_idx_new':[i for i in range(len(ad_rows))], 'row_idx_og':ad_rows})

In [10]:
ped_row_map.to_csv('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/pediatric/pat_map.csv',index=False)
ad_row_map.to_csv('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/adult/pat_map.csv',index=False)

In [31]:
# ad_list = [features[a].todense() for a in ad_rows]
# ped_list = [features[p].todense() for p in ped_rows]

In [11]:
ped_feats = [features.getrow(r) for r in ped_rows]

  (0, 2432)	1
  (0, 2436)	1
  (0, 3278)	1
  (0, 3279)	1
  (0, 3280)	1
  (0, 3281)	1
  (0, 3284)	1
  (0, 3285)	1
  (0, 4103)	1
  (0, 4112)	1
  (0, 8204)	1
  (0, 8205)	1
  (0, 8206)	1
  (0, 8207)	1
  (0, 8210)	1
  (0, 8211)	1
  (0, 8212)	1
  (0, 8216)	1
  (0, 8218)	1
  (0, 8220)	1
  (0, 8221)	1
  (0, 8222)	1
  (0, 19125)	1
  (0, 87746)	1
  (0, 87788)	1
  :	:
  (0, 235062)	1
  (0, 235066)	1
  (0, 235309)	1
  (0, 235339)	1
  (0, 235362)	1
  (0, 235445)	1
  (0, 235613)	1
  (0, 235629)	1
  (0, 239005)	1
  (0, 239038)	1
  (0, 239039)	1
  (0, 239040)	1
  (0, 239041)	1
  (0, 239043)	1
  (0, 239044)	1
  (0, 239045)	1
  (0, 239046)	1
  (0, 239047)	1
  (0, 239048)	1
  (0, 239050)	1
  (0, 239052)	1
  (0, 239055)	1
  (0, 239057)	1
  (0, 239058)	1
  (0, 239059)	1


In [12]:
pkl_file = open('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/pediatric/full.txt','wb')
pickle.dump(ped_list, pkl_file)

In [13]:
pkl_file = open('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/adult/full.txt','wb')
pickle.dump(ad_list, pkl_file)

In [ ]:
#sum_ped_list = np.sum(np.array(ped_list), axis=1)
#sum_ad_list = ad_list.sum(axis=1)

In [15]:
bin_ped_feat_list = [0 if i==0 else 1 for i in sum_ped_list]
bin_ad_feat_list = [0 if i==0 else 1 for i in sum_ad_list]

/tmp/ipykernel_2349198/936436005.py:1: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  bin_ped_feat_list = [0 if i==0 else 1 for i in sum_ped_list]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all().

In [37]:
print(np.sum(bin_ped_feat_list))
print(np.sum(bin_ad_feat_list))

112439
222242


In [13]:
feat_count_dict = {
    'neither': 0,
    'both': 0,
    'pediatric': 0,
    'adult': 0
}
ped_feat_indices = []
ad_feat_indices = []
shared_feat_indices = []
for i in range(len(bin_ped_feat_list)):
    ped = bin_ped_feat_list[i]
    ad = bin_ad_feat_list[i]
    if ped == 0 and ad == 0:
        feat_count_dict['neither'] = feat_count_dict['neither'] + 1
    elif ped == 1 and ad == 0:
        feat_count_dict['pediatric'] = feat_count_dict['pediatric'] + 1
        ped_feat_indices.append(i)
    elif ped == 0 and ad == 1:
        feat_count_dict['adult'] = feat_count_dict['adult'] + 1
        ad_feat_indices.append(i)
    elif ped == 1 and ad == 1:
        feat_count_dict['both'] = feat_count_dict['both'] + 1
        shared_feat_indices.append(i)

In [42]:
for k in feat_count_dict.keys():
    print(f'{k}:{feat_count_dict[k]}')

neither:9941
both:105444
pediatric:6995
adult:116557


In [40]:
pkl_file = open('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/adult/feat_used.txt','wb')
pickle.dump(bin_ad_feat_list, pkl_file)
pkl_file.close()
pkl_file = open('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/pediatric/feat_used.txt','wb')
pickle.dump(bin_ped_feat_list, pkl_file)
pkl_file.close()

In [10]:
pkl_file = open('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/adult/feat_used.txt','rb')
bin_ad_feat_list = pickle.load(pkl_file)
pkl_file.close()
pkl_file = open('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/pediatric/feat_used.txt','rb')
bin_ped_feat_list = pickle.load(pkl_file)
pkl_file.close()

In [14]:
pd.DataFrame({'feat_indices':ad_feat_indices}).to_csv('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/only_adult_feats.csv',index=False)
pd.DataFrame({'feat_indices':ped_feat_indices}).to_csv('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/only_pediatric_feats.csv',index=False)
pd.DataFrame({'feat_indices':shared_feat_indices}).to_csv('/local-scratch/nigam/projects/jlemmon/transfer_learning/experiments/data/bin_features/shared_feats.csv',index=False)